**Colab Link URL**
- https://colab.research.google.com/drive/1CKfHxFwzxJ3NfrqTBsgtulWAGDnzHe5M?usp=sharing

이 노트북은 [Google Colab](https://colab.research.google.com/)에서 실행하시기에 최적화되어있습니다.
상기 URL을 이용해서 코드를 실행 해보세요.

In [1]:
!pip install -q langchain langgraph langchain-community langchain-openai faiss-cpu pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 5.8 MB/s eta 0:00:00


In [2]:
import os
from typing import List, Dict, TypedDict

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langgraph.graph import StateGraph, END
from google.colab import userdata

In [3]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [4]:
# LangGraph 상태 정의
# 루프를 통해 전달될 데이터 구조를 정의
class GraphState(TypedDict):
    question: str
    documents: List[Document]
    answer: str

In [6]:
# 가상의 기술 문서를 지식 베이스로 사용
rag_documents = [
    Document(
        page_content="ACA의 동적 계획 수립 모듈은 '리플렉션(Reflection)' 메커니즘을 통해 작동합니다. 이 메커니즘은 실패 경험으로부터 학습하고, 그 원인을 분석하여 다음 행동 계획을 최적화하는 과정을 포함합니다.",
        metadata={"source": "ACA_Architecture_v2.pdf"}
    ),
    Document(
        page_content="ACA의 장기 기억 장치는 벡터 데이터베이스를 활용하여 과거의 성공 및 실패 경험을 저장하고 검색합니다.",
        metadata={"source": "ACA_Memory_v2.pdf"}
    ),
]
vectorstore = FAISS.from_documents(rag_documents, OpenAIEmbeddings(api_key=OPENAI_API_KEY))
retriever = vectorstore.as_retriever()

In [7]:
llm = ChatOpenAI(
    model="gpt-5-mini",
    api_key=OPENAI_API_KEY,
)

In [8]:
# 문서 검색 노드
def retrieve_node(state: GraphState) -> GraphState:
    print("--- [노드 실행] 문서 검색 ---")
    documents = retriever.invoke(state["question"])
    return {"documents": documents}

In [9]:
# 답변 생성 노드
def generate_node(state: GraphState) -> GraphState:
    print("--- [노드 실행] 답변 생성 ---")
    rag_chain = ChatPromptTemplate.from_template(
        "다음 문맥 정보를 바탕으로 질문에 답변하세요:\n\n문맥: {context}\n\n질문: {question}"
    ) | llm | StrOutputParser()

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    answer = rag_chain.invoke({"context": format_docs(state["documents"]), "question": state["question"]})
    return {"answer": answer}

In [10]:
# 문서 비판(Grade) 노드
def grade_documents_node(state: GraphState) -> GraphState:
    print("--- [노드 실행] 검색된 문서 평가 ---")
    grader_chain = ChatPromptTemplate.from_template(
        """당신은 검색된 문서가 사용자의 질문에 답변하기에 충분한 정보를 담고 있는지 평가하는 전문가입니다.
        'yes' 또는 'no'로만 답변해주세요.

        검색된 문서: {documents_text}
        사용자 질문: {question}"""
    ) | llm | StrOutputParser()

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    result = grader_chain.invoke({"question": state["question"], "documents_text": format_docs(state["documents"])})
    print(f"평가 결과: {result}")

    if "yes" in result.lower():
        return {"documents": state["documents"]}
    else:
        return {"documents": []}

In [11]:
# 문서 평가 결과에 따라 분기
def decide_to_generate(state: GraphState) -> str:
    print("--- [엣지 결정] 답변 생성 여부 ---")
    if not state["documents"]:
        print("결론: 검색된 문서의 관련성이 낮아 종료합니다.")
        return "end"
    else:
        print("결론: 검색된 문서의 관련성이 높아 답변을 생성합니다.")
        return "generate"

In [12]:
# 그래프 구성
workflow = StateGraph(GraphState)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("grade_documents", grade_documents_node)
workflow.add_node("generate", generate_node)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges("grade_documents", decide_to_generate, {"generate": "generate", "end": END})
workflow.add_edge("generate", END)
app = workflow.compile()

In [13]:
# 사례 1: 관련성 높은 질문 (성공 케이스)
print("--- [사례 1: 성공 케이스] ---")
question_yes = "ACA의 동적 계획 수립은 어떻게 작동하나요?"
print(f"질문: {question_yes}\n")
result_yes = app.invoke({"question": question_yes})
print("\n--- 최종 결과 (사례 1) ---")
print(result_yes.get("answer", "답변 생성에 실패했습니다."))

--- [사례 1: 성공 케이스] ---
질문: ACA의 동적 계획 수립은 어떻게 작동하나요?

--- [노드 실행] 문서 검색 ---
--- [노드 실행] 검색된 문서 평가 ---
평가 결과: yes
--- [엣지 결정] 답변 생성 여부 ---
결론: 검색된 문서의 관련성이 높아 답변을 생성합니다.
--- [노드 실행] 답변 생성 ---

--- 최종 결과 (사례 1) ---
ACA의 동적 계획 수립은 다음과 같은 순환적 프로세스로 작동합니다.

1. 실패 감지 및 기록  
   - 행동 실행 중 실패(또는 기대와 다른 결과)를 감지하면 그 경험을 벡터 형태로 인코딩하여 장기 기억에 기록합니다.

2. 유사 경험 검색(벡터 DB 활용)  
   - 벡터 데이터베이스에서 현재 상황과 유사한 과거의 성공·실패 사례를 유사도 검색으로 빠르게 불러옵니다.

3. 리플렉션(원인 분석)  
   - 불러온 사례들과 현재 실패를 비교·분석하여 실패의 원인(환경 요인, 전략·파라미터 문제 등)을 도출합니다.

4. 계획 최적화 및 재생성  
   - 분석 결과를 바탕으로 다음 행동 계획을 조정·재생성합니다(전략 변경, 파라미터 튜닝, 대체 행동 선택 등).

5. 실행 및 피드백 루프  
   - 수정된 계획을 실행하고 결과를 모니터링하여 성공이면 강화, 실패면 다시 기록하여 반복 학습합니다.

요약하면, ACA는 실패를 벡터형태로 장기기억에 저장·검색하고, 리플렉션을 통해 원인을 분석한 뒤 그 분석 결과로 다음 행동 계획을 최적화하는 반복적 학습-계획 수립 구조를 가집니다.


In [14]:
# 사례 2: 관련성 낮은 질문 (실패 방지 케이스)
print("--- [사례 2: 실패 방지 케이스] ---")
question_no = "음악 스트리밍 서비스의 역사에 대해 알려줘."
print(f"질문: {question_no}\n")
result_no = app.invoke({"question": question_no})
print("\n--- 최종 결과 (사례 2) ---")
print(result_no.get("answer", "답변 생성에 실패했습니다."))

--- [사례 2: 실패 방지 케이스] ---
질문: 음악 스트리밍 서비스의 역사에 대해 알려줘.

--- [노드 실행] 문서 검색 ---
--- [노드 실행] 검색된 문서 평가 ---
평가 결과: no
--- [엣지 결정] 답변 생성 여부 ---
결론: 검색된 문서의 관련성이 낮아 종료합니다.

--- 최종 결과 (사례 2) ---
답변 생성에 실패했습니다.
